# Wikipedia search

# Get all the data from the wikipedia

In [10]:
import wikipedia
import requests
from bs4 import BeautifulSoup
import random

# Infrmation of the book
author = "Ruge Sophus"
title = "Zeitalter Entdeckungen"
old_date = 1828 

wikipedia.set_lang("de")

# Search for the possible wiki-websites about the author
search = wikipedia.search(author)
search += wikipedia.search(title)

# Function to get the html of the author, for easier usage in the for loop
def gethtmlbyauthor(author):
    # replace the spaces with the _ so it can be used in the url
    author.replace(" ", "_")
    response = requests.get(
		url="https://de.wikipedia.org/wiki/" + author,
	)
    # get the content of the website
    soup = BeautifulSoup(response.content, 'html.parser')
    content = soup.find_all('div', class_='mw-content-ltr').__str__()
    return content

# the content of the websites that include a part of the title of the book
found = []
for s in search:
    # get the content of the website
    try:
        content = wikipedia.page(s).content
    except:
        content = gethtmlbyauthor(s)
    for p in title.split(" "):
        # check if a part of the title is in the content of the website
        if p in content:
            # add the content to the found list
            found.append(content)
            # print the deatils of the search
            print("Search: " + s + "\nWord in Text: " + p)
            break

print()

margin = 500
n = -1
# list to store the found sentences with the title
promts = []
for f in found:
    # look for the position of the title in the content
    for p in title.split(" "):
        n = f.find(p)
        if n == -1:
            continue
        else:
            print(p)
            break
    # get the sentence and text around the title
    if n < margin:
        text_to_check = f[:n+margin]
    else:
        text_to_check = f[n-margin:n+margin]
    # print the sentence
    print("text (" + str(n) + "):\n" + text_to_check + "\n")
    # add the sentence to the list
    promts.append(text_to_check)

Search: Sophus Ruge
Word in Text: Zeitalter
Search: Friedrich Ruge
Word in Text: Zeitalter
Search: Zeitalter der Entdeckungen
Word in Text: Zeitalter
Search: Entdeckung
Word in Text: Zeitalter
Search: Padrão dos Descobrimentos
Word in Text: Zeitalter
Search: Entdeckung Amerikas 1492
Word in Text: Zeitalter
Search: Goldenes Zeitalter
Word in Text: Zeitalter
Search: Ära
Word in Text: Zeitalter
Search: Entdeckung Amerikas
Word in Text: Zeitalter
Search: Mittelalter
Word in Text: Zeitalter
Search: Frühe Neuzeit
Word in Text: Zeitalter
Search: Expo 92
Word in Text: Zeitalter

Zeitalter
text (2504):
hie, die er bis zu seiner krankheitsbedingten Beurlaubung im Jahr 1903 innehatte.
Im Jahr 1877 war er einer der Gründer des Gebirgsvereins für die sächsisch-böhmische Schweiz, dessen erster Vorsitzender bis 1885 und späteres Ehrenmitglied er war. Insbesondere in den Anfangsjahren schrieb er Beiträge für das 1878 von ihm mitbegründete Vereinsorgan Über Berg und Thal.
Seine wissenschaftliche Haupta

# Go through the text and see if there is a link for the book.

In [18]:
for p in promts:
    for s in title.split(" "):
        n = p.find(s)
        if n != -1:
            print(str(n) + ": " + s)
            break

500: Jenny
500: Jenny
500: Jenny
500: Roman
500: Berliner
500: Gesellschaft
500: Jenny
500: Gesellschaft
500: Jenny
500: Jenny
500: Jenny
500: Jenny
500: Jenny
500: Jenny
500: Jenny
500: Jenny
500: Jenny


In [3]:
cont = wikipedia.page("Frau Jenny Treibel").content
n = cont.find("Jenny")
print(n)
if n < 100:
    print(cont[:n+100])
else:
    print(cont[n-100:n+100])

5
Frau Jenny Treibel oder „Wo sich Herz zum Herzen find’t“ ist ein Roman Theodor Fontanes. Von Januar bis A


# Clear the found text of any html

In [11]:
def clear_text(text):
    # removes the first part if it already starts within a html tag
    if text.find(">") < text.find("<"):
        text = text[text.find(">")+1:]

    html = False
    i = 0
    while i < len(text):
        if text[i] == '<':
            html = True
        # if inside a html tag, remove the character
        if html:
            if text[i] == '>':
                html = False
                text = text[:i] + "  " + text[i+1:]
            else:
                text = text[:i] + text[i+1:]
                i -= 1
        i += 1
    return text

for i in range(0,len(promts)):
    promts[i] = clear_text(promts[i])
    print(promts[i] + "\n")

hie, die er bis zu seiner krankheitsbedingten Beurlaubung im Jahr 1903 innehatte.
Im Jahr 1877 war er einer der Gründer des Gebirgsvereins für die sächsisch-böhmische Schweiz, dessen erster Vorsitzender bis 1885 und späteres Ehrenmitglied er war. Insbesondere in den Anfangsjahren schrieb er Beiträge für das 1878 von ihm mitbegründete Vereinsorgan Über Berg und Thal.
Seine wissenschaftliche Hauptarbeiten bezogen sich auf die Geschichte der Geo- und Kartographie, der Schwerpunkt lag dabei auf dem Zeitalter der Entdeckungen und der Darstellung der kartographischen Erschließung der Neuen Welt. Durch eine stoffliche Systematisierung formte er die Geographie dabei zu einem akademischen Lehrfach. Außerdem war er mit eingehenden Quellenstudien maßgeblich an der Überarbeitung und Neubearbeitung mehrerer Lehrbücher beteiligt. Ab 1878 überarbeitete er Die Geschichte der Erdkunde (begründet von Oscar Ferdinand Peschel) und ab 1887 Die Erdbeschreibung (begründet von Franz Heinrich Ungewitter). In d

# The model that searches for the year in the found sentences

loading the model

In [12]:
model_id = 'textminr/ner-multilingual-bert'

from transformers import pipeline
classifier = pipeline(
  'ner',
  model=model_id,
  aggregation_strategy='simple'
)

using the model

In [13]:
responses = []
for i in range(len(promts)):
    # model looks for the correct date in the text
    responses.append(classifier(promts[i]))
    print(str(i) + ":")
    # only the dates are printed, because the model also looks for other entities
    for r in responses[i]:
       print(r)

0:
{'entity_group': 'DATE', 'score': 0.9990601, 'word': '1877', 'start': 90, 'end': 94}
{'entity_group': 'DATE', 'score': 0.8586997, 'word': '1885', 'start': 207, 'end': 211}
{'entity_group': 'AUTHOR', 'score': 0.9116224, 'word': 'Oscar Ferdinand Peschel', 'start': 896, 'end': 919}
{'entity_group': 'AUTHOR', 'score': 0.89329845, 'word': 'Franz Heinrich Ungewitter', 'start': 968, 'end': 993}
1:
{'entity_group': 'DATE', 'score': 0.9886049, 'word': '1985', 'start': 46, 'end': 50}
{'entity_group': 'AUTHOR', 'score': 0.6579502, 'word': 'Winkler', 'start': 53, 'end': 60}
{'entity_group': 'AUTHOR', 'score': 0.52080005, 'word': 'Bochum', 'start': 62, 'end': 68}
{'entity_group': 'DATE', 'score': 0.99925953, 'word': '2005', 'start': 69, 'end': 73}
{'entity_group': 'AUTHOR', 'score': 0.59526086, 'word': '##11', 'start': 85, 'end': 87}
{'entity_group': 'DATE', 'score': 0.50750935, 'word': '10', 'start': 158, 'end': 160}
{'entity_group': 'AUTHOR', 'score': 0.9948858, 'word': 'Berthold J. Sander - N